In [7]:
import pandas as pd

# Our sample data
sales_data = pd.DataFrame({
    'Region': ['North', 'North', 'South', 'South', 'North', 'South'],
    'Product': ['A', 'B', 'A', 'B', 'A', 'B'],
    'Quarter': ['Q1', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2'],
    'Sales': [1000, 1500, 1200, 1800, 1100, 1600],
    'Units': [50, 75, 60, 90, 55, 80]
})

In [8]:
# Total sales by region and product
sales_by_region = pivot_df(sales_data, 
                          index='Region', 
                          columns='Product', 
                          values='Sales', 
                          aggfunc='sum')
sales_by_region

Product,A,B
Region,,
North,2100,1500
South,1200,3400


In [9]:
# Average units sold by quarter and region
units_by_quarter = pivot_df(sales_data,
                           index='Quarter',
                           columns='Region', 
                           values='Units',
                           aggfunc='mean')
units_by_quarter

Region,North,South
Quarter,,
Q1,62.5,75.0
Q2,55.0,80.0


In [10]:
# Multiple aggregations with margins
summary = pivot_df(sales_data,
                  index='Region',
                  columns='Quarter',
                  values=['Sales', 'Units'],
                  aggfunc={'Sales': 'sum', 'Units': 'mean'},
                  margins=True)
summary

Sales                Units                 
Quarter      Q1      Q2   All     Q1    Q2        All
Region                                               
North    2500.0  1100.0  3600  62.50  55.0  60.000000
South    3000.0  1600.0  4600  75.00  80.0  76.666667
All      5500.0  2700.0  8200  68.75  67.5  68.333333